# A/B TESTING WITH MACHINE LEARNING
A/b testing compares the conversion rates of only two groups; Exposed group and the Control group. But in the real world, the situation and even the behaviours of online users is complex and more dynamic. Situations such as:

Online users spend a different amount of time in the Advertisement.
Users take different paths to the Ad.
Users use different devices and Also view the Ad on different dates.

With all these complexity, machine learning helps in generating insights from these complex systems.


### Loading Libraries we will use in the project

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [2]:
# import libraries
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import scipy.stats as stat
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import pydotplus
from sklearn import tree

ModuleNotFoundError: No module named 'sklearn.externals.six'

### Importing scripts & modules 

In [3]:
from helper import MyHelper
import utility
import plots
Helper = MyHelper()


### Loading the data

In [13]:
CSV_PATH = "../data/mydata.csv.csv"
data_frame = Helper.read_csv(CSV_PATH)

>> file read as csv


### Drop observations where users didn't respond to the questionaire

In [14]:
# drop columns where yes and no are both 0
drop_mask = data_frame.query('yes==0 and no==0') 
data_frame=data_frame.drop(drop_mask.index)
data_frame.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
16,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0
20,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0,1
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1,0
27,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,1


### Function for creating an awareness column


In [15]:
# Function for creating an awareness column
def label_awareness (row):
    if row['yes'] == 1 :
         return '1'
    if row['no'] == 1 :
        return '0'
def calculate_metrics(y_test, y_preds):
    
    accuracy = accuracy_score(y_test, y_preds)
    return accuracy


### Creating 'awarness' column

In [16]:
# apply awareness to each row
data_frame['awareness'] = data_frame.apply (lambda row: label_awareness(row), axis=1) 
data_frame.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,awareness
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,0
16,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0,1
20,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0,1,0
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1,0,1
27,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,1,0


### change categorical variables to numerical value


In [17]:
# change categorical variables to numerical value
lb = LabelEncoder()
data_frame['experiment'] = lb.fit_transform(data_frame['experiment'])
data_frame['browser'] = lb.fit_transform(data_frame['browser'])
data_frame['device_make'] = lb.fit_transform(data_frame['device_make'])
data_frame['date'] = lb.fit_transform(data_frame['date'])
data_frame['awareness'] = lb.fit_transform(data_frame['awareness'])


### Split data by browser and platform_os, and version each split as a new version of the data in dvc

In [18]:
broweser_df = data_frame[["experiment", "hour", "date", 'device_make', 'browser', 'awareness']]
platfrom_df = data_frame[["experiment", "hour", "date", 'device_make', 'platform_os', 'awareness']]

In [19]:
Helper.save_csv(broweser_df, "../data/mydata.csv.csv")

>> File Successfully Saved.!!!


,experiment,hour,date,device_make,browser,awareness
2,1,2,2,4,2,0
16,1,16,1,13,1,1
20,1,8,3,13,1,0
23,0,4,5,43,4,1
27,0,15,0,13,1,0
...,...,...,...,...,...,...
8059,1,21,2,13,1,1
8063,1,1,1,13,1,1
8064,0,7,6,13,1,0
8069,0,16,7,13,1,0


In [20]:
Helper.save_csv(platfrom_df, "../data/mydata.csv.csv")

>> File Successfully Saved.!!!


,experiment,hour,date,device_make,platform_os,awareness
2,1,2,2,4,6,0
16,1,16,1,13,6,1
20,1,8,3,13,6,0
23,0,4,5,43,6,1
27,0,15,0,13,6,0
...,...,...,...,...,...,...
8059,1,21,2,13,6,1
8063,1,1,1,13,6,1
8064,0,7,6,13,6,0
8069,0,16,7,13,6,0


## Different versions of the data tags
### ver1 - Original Data
### ver2 - Split by Browser
### ver3 - Split by Platform